In [2]:
#!pip install datasets

In [1]:
from datasets import load_dataset
tinyStories_ds = load_dataset("roneneldan/TinyStories")

c:\Users\Public\anaconda3\envs\torch-gpu-cuda12.5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from functools import partial
from torch.utils.data import DataLoader
import torch
import tiktoken
import sys

In [3]:
sys.path.insert(0, 'D:\LLM_Deeplearning.ai\SEBASTIAN_RASCHKA\Large_Language_Model_From_Scratch_Implementation\transformer_blocks')
sys.path.insert(1, 'D:\LLM_Deeplearning.ai\SEBASTIAN_RASCHKA\Large_Language_Model_From_Scratch_Implementation\gpt_Pretraining')

from text_generation import Text_Generation
#from gpt2 import GPT2

In [4]:
#Printing to find out the structure of the huggingface dataset:
print(tinyStories_ds)
print('Printing 10 records from the Train set : ')

i = 0
for ts in tinyStories_ds['train']:
    print(ts)
    
    if i > 10:
        break
    else:
        i = i + 1
        continue


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})
Printing 10 records from the Train set : 
{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}
{'text': 'Once upon a time, there was a little car name

In [14]:
class GPTCustomPretrainDataset:
    def __init__(self, tokenizer, input_data):
        self.data = input_data
        self.encoded_data = []
        self.total_tokens = 0

        for row in input_data:
            raw_text = row['text']
            encoded_text = tokenizer.encode(raw_text, allowed_special={"<|endoftext|>"})
            y = len(encoded_text)
            self.total_tokens = self.total_tokens + y
            self.encoded_data.append(encoded_text)

    def __len__(self):
        return len(self.encoded_data)
    
    def __getitem__(self, index):
        return self.encoded_data[index]
    
    def get_total_tokens(self):
        return self.total_tokens

In [15]:
def custom_collate_pretrain(batch, pad_token = 50256, device = 'cpu', max_seq_length = None):

    input_list, target_list = [], []
    batch_max_length = max([len(item)+1 for item in batch])

    for row in batch:

        tokens = row.copy()
        tokens = tokens + [pad_token]

        padded_tokens = tokens + [pad_token] * (batch_max_length - len(tokens))

        inputs = torch.tensor(padded_tokens[:-1])
        targets = torch.tensor(padded_tokens[1:])

        if max_seq_length is not None:
            inputs = inputs[:max_seq_length]
            targets = targets[:max_seq_length]


        input_list.append(inputs)
        target_list.append(targets)

    inputs_tensor = torch.stack(input_list).to(device)
    targets_tensor = torch.stack(target_list).to(device)

    return inputs_tensor, targets_tensor

In [ ]:
def GPTCustomPretrainDataloader(data, tokenizer='tiktoken', pad_token=None, device = 'cpu', max_seq_length = None, batch_size=8,
                                shuffle=True, drop_last=True,num_workers=0):

    if tokenizer == 'tiktoken':
        tokenizer = tiktoken.get_encoding('gpt2')

    #Get the last token id of the tokenizer selected:
    if pad_token is None:
        pad_token = tokenizer.encode('<|endoftext|>', allowed_special='all')[0]

    torch.manual_seed(123)

    c_collate_pretrain = partial(custom_collate_pretrain, device=device, max_seq_length=max_seq_length , pad_token = pad_token)

    dataset = GPTCustomPretrainDataset(tokenizer=tokenizer, input_data=data)
    total_tokens = dataset.get_total_tokens()
    print('Total Tokens : ', total_tokens)

    #Create the dataloader with the dataset
    custom_dataloader = DataLoader(dataset,batch_size=batch_size, collate_fn=c_collate_pretrain, 
                                   shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return custom_dataloader, total_tokens

In [17]:
train_df = tinyStories_ds['train']
val_df = tinyStories_ds['validation']
print(f'Total records in Train Data: {len(train_df)} , Validation Data : {len(val_df)}')

Total records in Train Data: 2119719 , Validation Data : 21990


In [9]:
train_df[1]

{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'}

In [37]:
i=0
for row in train_df:
    c = row['text']
    print(c)
    print(row.keys())
    
    if i > 5:
        break
    else:
        i = i + 1
        print('---------')
        continue

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.
dict_keys(['text'])
---------
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had ma

In [18]:
inputs_1 = ([0, 1, 2, 3, 4]) #6, 6 --> 0 pad token in x
inputs_2 = ([5, 6]) #3 pad token in x
inputs_3 = ([7, 8, 9]) #2 pad token in x


batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print('Checking Collate Function Performance :')

print('Without Instruction Masking :: ')
i, t = custom_collate_pretrain(batch=batch)
print('Input: ', i)
print('Target: ', t)
print('------------------')

Checking Collate Function Performance :
Without Instruction Masking :: 
Input:  tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
Target:  tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256, 50256, 50256, 50256],
        [    8,     9, 50256, 50256, 50256]])
------------------


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device Available: ', device)

Device Available:  cuda


In [20]:
def print_loader(loader):
    i=0
    for inputs, targets in loader:
        print(inputs.shape, targets.shape)
        if i > 10:
            break
        else:
            i += 1

In [46]:
train_loader = GPTCustomPretrainDataloader(train_df, device=device, max_seq_length=64)
print('Number of training batches :: ', len(train_loader))

print('Train Loader ::')
print_loader(train_loader)
print('--------------------------')

val_loader = GPTCustomPretrainDataloader(val_df, device=device, max_seq_length=64)
print('Number of validation batches :: ', len(val_loader))

print('Validation Loader ::')
print_loader(val_loader)
print('--------------------------')

dataset got
Number of training batches ::  264964
Train Loader ::
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
--------------------------
dataset got
Number of validation batches ::  2748
Validation Loader ::
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 64])
torch.Size([8, 64]) torch.Size([8, 6

In [47]:
train_loader = GPTCustomPretrainDataloader(train_df, device=device, max_seq_length=None)
print('Number of training batches :: ', len(train_loader))

print('Train Loader ::')
print_loader(train_loader)
print('--------------------------')

val_loader = GPTCustomPretrainDataloader(val_df, device=device, max_seq_length=None)
print('Number of validation batches :: ', len(val_loader))

print('Validation Loader ::')
print_loader(val_loader)
print('--------------------------')

dataset got
Number of training batches ::  264964
Train Loader ::
torch.Size([8, 263]) torch.Size([8, 263])
torch.Size([8, 577]) torch.Size([8, 577])
torch.Size([8, 338]) torch.Size([8, 338])
torch.Size([8, 271]) torch.Size([8, 271])
torch.Size([8, 232]) torch.Size([8, 232])
torch.Size([8, 761]) torch.Size([8, 761])
torch.Size([8, 258]) torch.Size([8, 258])
torch.Size([8, 319]) torch.Size([8, 319])
torch.Size([8, 477]) torch.Size([8, 477])
torch.Size([8, 269]) torch.Size([8, 269])
torch.Size([8, 382]) torch.Size([8, 382])
torch.Size([8, 562]) torch.Size([8, 562])
--------------------------
dataset got
Number of validation batches ::  2748
Validation Loader ::
torch.Size([8, 668]) torch.Size([8, 668])
torch.Size([8, 522]) torch.Size([8, 522])
torch.Size([8, 574]) torch.Size([8, 574])
torch.Size([8, 240]) torch.Size([8, 240])
torch.Size([8, 201]) torch.Size([8, 201])
torch.Size([8, 298]) torch.Size([8, 298])
torch.Size([8, 295]) torch.Size([8, 295])
torch.Size([8, 681]) torch.Size([8, 68